In [27]:
import uproot
import awkward as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from tqdm.notebook import tqdm
from collections import Counter

LUMI_NUMBER = 2000
SEQ_LENGTH = 3
COLLIDING_BUNCHES = False

In [28]:
if COLLIDING_BUNCHES:
    filepath = f"data-sequences/output_{LUMI_NUMBER}_seq{SEQ_LENGTH}.root"

else:
    filepath = f"data-sequences/output_{LUMI_NUMBER}_seq{SEQ_LENGTH}_nc.root"

data = uproot.open(filepath + ":L1BMTFStubSequences").arrays()
df = ak.to_dataframe(data)
df.head()

,,nL1BMTFStub,L1BMTFStub_hwQual,L1BMTFStub_hwPhi,L1BMTFStub_hwPhiB,L1BMTFStub_hwEta,L1BMTFStub_hwQEta,L1BMTFStub_wheel,L1BMTFStub_sector,L1BMTFStub_station,orbitNumber,bunchCrossing,sequenceIndex
entry,subentry,,,,,,,,,,,,
0,0,1,3,1557,-90,0,0,0,3,4,524028021,3547,0
1,0,1,3,1557,-90,0,0,0,3,4,524028021,3548,0
2,0,1,3,1557,-90,0,0,0,3,4,524028021,3549,0
3,0,1,5,751,-43,0,0,-1,3,4,524028336,2312,1
4,0,2,6,1034,-106,8,0,-1,3,1,524028336,2313,1


In [29]:
dfgb = df.groupby("sequenceIndex")

index = 12

print(f"Index: {index}")
dfg = dfgb.get_group(index)
dfg_selected = dfg[["bunchCrossing", "L1BMTFStub_wheel", "L1BMTFStub_station"]]
print(dfg_selected)

print(f"\n\nSequence: {index}")
for sector, dfsector in dfg.groupby("L1BMTFStub_sector"):
    print(f'\tSector: {sector}, Wheel: {dfsector["L1BMTFStub_wheel"].values}, Station: {dfsector["L1BMTFStub_station"].values}, BX: {dfsector["bunchCrossing"].values}')

Index: 12
                bunchCrossing  L1BMTFStub_wheel  L1BMTFStub_station
entry subentry                                                     
36    0                   416                 1                   1
37    0                   417                 1                   1
38    0                   418                 1                   1


Sequence: 12
	Sector: 8, Wheel: [1 1 1], Station: [1 1 1], BX: [416 417 418]


In [30]:
[
    tuple(el.item() for el in key) 
    for key in dfg_selected.groupby(["bunchCrossing", "L1BMTFStub_station"]).indices
]

[(416, 1), (417, 1), (418, 1)]

In [31]:
def getSameBxTargetCollections(bxs):
    target_pairs = [
        [(bx, station) for station in range(ii, ii + SEQ_LENGTH)]
        for bx in bxs
        for ii in range(1, 4 - SEQ_LENGTH + 2)
    ]
    
    return target_pairs


# interesting ones !!!
def getMultiBxTargetCollections(bxs):
    target_pairs = [
        [(bx, station) for bx, station in zip(bxs, range(ii, ii + SEQ_LENGTH))]
        for ii in range(1, 4 - SEQ_LENGTH + 2)
    ]

    return target_pairs

def getInverseMultiBxTargetCollections(bxs):
    target_pairs = [
        [(bx, station) for bx, station in zip(bxs, range(ii + SEQ_LENGTH, ii, -1))]
        for ii in range(0, 4 - SEQ_LENGTH + 1)
    ]

    return target_pairs

In [32]:
multi_bx_selection = {}
inverse_multi_bx_selection = {}

for idx_seq, df_seq in tqdm(df.groupby("sequenceIndex")):
    pairs = [
        tuple(el.item() for el in key) 
        for key in df_seq.groupby(["bunchCrossing", "L1BMTFStub_station"]).indices
    ]

    multi_bx_collections = getMultiBxTargetCollections(df_seq["bunchCrossing"].unique().tolist())

    for tps in multi_bx_collections:
        if np.all([target_pair in pairs for target_pair in tps]):
            """
            Here we keep only the entries of the sequence sub-dataframe that 
            correspond to the identified (bx, station) pairs.
            This allows us then to groupby wheel and keep only the wheels that are
            at most distant 1 to one another.
            """
            mi = pd.MultiIndex.from_tuples(tps, names=["bunchCrossing", "L1BMTFStub_station"])
            df_temp = df_seq[
                df_seq.set_index(["bunchCrossing", "L1BMTFStub_station"]).index.isin(mi)
            ]

            if df_temp["L1BMTFStub_wheel"].max() - df_temp["L1BMTFStub_wheel"].min() > 1:
                if df_temp[df_temp["L1BMTFStub_wheel"] < df_temp["L1BMTFStub_wheel"].max()]["bunchCrossing"].nunique() < SEQ_LENGTH:
                    if df_temp[df_temp["L1BMTFStub_wheel"] > df_temp["L1BMTFStub_wheel"].min()]["bunchCrossing"].nunique() < SEQ_LENGTH:
                        continue

                    else:
                        df_temp = df_temp[df_temp["L1BMTFStub_wheel"] > df_temp["L1BMTFStub_wheel"].min()]
                        multi_bx_selection[idx_seq] = df_temp

                else:
                    df_temp = df_temp[df_temp["L1BMTFStub_wheel"] < df_temp["L1BMTFStub_wheel"].max()]
                    multi_bx_selection[idx_seq] = df_temp

            else:
                multi_bx_selection[idx_seq] = df_temp

    inverse_multi_bx_collections = getInverseMultiBxTargetCollections(df_seq["bunchCrossing"].unique().tolist())

    for tps in inverse_multi_bx_collections:
        if np.all([target_pair in pairs for target_pair in tps]):
            """
            Here we keep only the entries of the sequence sub-dataframe that 
            correspond to the identified (bx, station) pairs.
            This allows us then to groupby wheel and keep only the wheels that are
            at most distant 1 to one another.
            """
            mi = pd.MultiIndex.from_tuples(tps, names=["bunchCrossing", "L1BMTFStub_station"])
            df_temp = df_seq[
                df_seq.set_index(["bunchCrossing", "L1BMTFStub_station"]).index.isin(mi)
            ]

            if df_temp["L1BMTFStub_wheel"].max() - df_temp["L1BMTFStub_wheel"].min() > 1:
                if df_temp[df_temp["L1BMTFStub_wheel"] < df_temp["L1BMTFStub_wheel"].max()]["bunchCrossing"].nunique() < SEQ_LENGTH:
                    if df_temp[df_temp["L1BMTFStub_wheel"] > df_temp["L1BMTFStub_wheel"].min()]["bunchCrossing"].nunique() < SEQ_LENGTH:
                        continue

                    else:
                        df_temp = df_temp[df_temp["L1BMTFStub_wheel"] > df_temp["L1BMTFStub_wheel"].min()]
                        inverse_multi_bx_selection[idx_seq] = df_temp

                else:
                    df_temp = df_temp[df_temp["L1BMTFStub_wheel"] < df_temp["L1BMTFStub_wheel"].max()]
                    inverse_multi_bx_selection[idx_seq] = df_temp

            else:
                inverse_multi_bx_selection[idx_seq] = df_temp

  0%|          | 0/350 [00:00<?, ?it/s]

In [33]:
df_tot_mbs = pd.concat(list(multi_bx_selection.values()))
df_tot_imbs = pd.concat(list(inverse_multi_bx_selection.values()))

In [34]:
if COLLIDING_BUNCHES:
    pkl_mbs = f"df_mbs_output_{LUMI_NUMBER}_seq{SEQ_LENGTH}.pkl"
    pkl_imbs = f"df_imbs_output_{LUMI_NUMBER}_seq{SEQ_LENGTH}.pkl"

else:
    pkl_mbs = f"df_mbs_output_{LUMI_NUMBER}_seq{SEQ_LENGTH}_nc.pkl"
    pkl_imbs = f"df_imbs_output_{LUMI_NUMBER}_seq{SEQ_LENGTH}_nc.pkl"

df_tot_mbs.to_pickle("selected-sequences/" + pkl_mbs)
df_tot_imbs.to_pickle("selected-sequences/" + pkl_imbs)